# Bol.com Web Scraping

In [1]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

import time

from tqdm import tqdm

In [2]:
categories = ['laptops', 'desktops', 'monitoren', 'randapparatuur', 'netwerk-&-internet', 'tassen-&-hoezen', 'printers-&-supplies', 'pc-gaming-winkel', 'software', 'dataopslag', 'beamers', 'componenten']

In [3]:
print(len(categories))

12


In [4]:
all_products = []

for i in (categories):
    for j in tqdm(range(1, 501)):
        response = requests.get(f'https://www.bol.com/nl/s/?page={j}&searchtext={i}&view=list').content
        soup = BeautifulSoup(response, 'lxml')
        products = soup.find_all(class_='product-item--row js_item_root')
        all_products.extend(products)
        time.sleep(2)

100%|██████████| 500/500 [21:41<00:00,  2.60s/it]


In [5]:
lst = []

for item in all_products:
    if item.find(class_='product-prices').getText() == '\nNiet leverbaar\n':
        price = None
    elif item.find(class_='star-rating')['title'] == 'Er zijn nog geen beoordelingen voor dit product.':
        score = None
    elif item.find(class_='star-rating')['title'] == 'Er zijn  nog geen beoordelingen voor dit product.':
        review = None
    else:
        product = item.find(class_='product-title--inline').a.getText()
        price = float(item.meta['content'])
        score = item.find(class_='star-rating')['title']
        review = item.find(class_='star-rating')['data-count']
        lst.append([product, price, score, review])

In [6]:
print(lst[:10])

[['ASUS Chromebook C523NA-A20209 - Chromebook - 15.6 inch', 265.95, 'klantbeoordeling: 4.4 van de 5 sterren', '66'], ['Lenovo Ideapad L340-15IRH 81LK0150MH - Gaming Laptop - 15.6 Inch', 729.0, 'klantbeoordeling: 4.6 van de 5 sterren', '19'], ['Apple MacBook Air (April, 2020) MWTJ2N/A - 13.3 inch - Intel Core i3 - 256 GB - Spacegrijs', 979.0, 'klantbeoordeling: 4.8 van de 5 sterren', '13'], ['Asus Chromebook C523NA-EJ0186 - Chromebook - 15.6 Inch', 299.0, 'klantbeoordeling: 4.4 van de 5 sterren', '66'], ['HP 17-by3740nd - Laptop - 17.3 Inch', 749.0, 'klantbeoordeling: 4.7 van de 5 sterren', '10'], ['ASUS TUF Gaming FX505GT-BQ166T - Gaming Laptop - 15.6 inch', 749.0, 'klantbeoordeling: 4.3 van de 5 sterren', '11'], ['Lenovo Ideapad S330 Chromebook 81JW0009MH - Chromebook - 14 Inch', 217.73, 'klantbeoordeling: 4.3 van de 5 sterren', '40'], ['ASUS VivoBook 15 K512JP-BQ281T - Laptop - 15.6 Inch', 849.0, 'klantbeoordeling: 4.5 van de 5 sterren', '22'], ['Lenovo Ideapad Slim 1-14AST-05 81VS00

In [7]:
products_downloaded = len(lst)

In [8]:
df = pd.DataFrame(lst, columns=['Product', 'Price', 'Score', 'Reviews'])

In [9]:
len(df)

30206

In [10]:
df['Price'] = df['Price'].dropna()

In [11]:
len(df)

30206

In [12]:
df = df.drop_duplicates()

In [13]:
len(df)

6706

In [14]:
df = df.reset_index(drop=True)

In [15]:
df['Score'] = df['Score'].str.strip('klantbeoordeling: ')

In [16]:
df['Score'] = df['Score'].str[:3]

In [17]:
df['Score'] = df['Score'].astype(float)

In [18]:
df['Reviews'] = df['Reviews'].astype(int)

In [19]:
df.head(10)

,Product,Price,Score,Reviews
0,ASUS Chromebook C523NA-A20209 - Chromebook - 1...,265.95,4.4,66
1,Lenovo Ideapad L340-15IRH 81LK0150MH - Gaming ...,729.00,4.6,19
2,"Apple MacBook Air (April, 2020) MWTJ2N/A - 13....",979.00,4.8,13
3,Asus Chromebook C523NA-EJ0186 - Chromebook - 1...,299.00,4.4,66
4,HP 17-by3740nd - Laptop - 17.3 Inch,749.00,4.7,10
5,ASUS TUF Gaming FX505GT-BQ166T - Gaming Laptop...,749.00,4.3,11
6,Lenovo Ideapad S330 Chromebook 81JW0009MH - Ch...,217.73,4.3,40
7,ASUS VivoBook 15 K512JP-BQ281T - Laptop - 15.6...,849.00,4.5,22
8,Lenovo Ideapad Slim 1-14AST-05 81VS006SMH - La...,150.67,3.6,24
9,ASUS Chromebook C523NA-EJ0055 - Chromebook - 1...,245.95,4.4,66


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6706 entries, 0 to 6705
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Product  6706 non-null   object 
 1   Price    6706 non-null   float64
 2   Score    6706 non-null   float64
 3   Reviews  6706 non-null   int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 209.7+ KB


In [21]:
print('Products Downloaded:', products_downloaded, '\nDataset Size:', len(df))

Products Downloaded: 30206 
Dataset Size: 6706


In [22]:
df.to_csv('../data/processed/bol.csv')